In [1]:
!pip3 install pyspark

You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
from pyspark import SparkContext

In [3]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
import os

spark = SparkSession.builder \
    .master("local") \
    .appName("Test it 2") \
    .getOrCreate()
sc = spark.sparkContext


In [4]:
spark

### DataFrames

DataFrames gives a schema view of data basically, it is an abstraction. In dataframes, view of data is organized as columns with column name and types info. In addition, we can say data in dataframe is as same as the table in relational database.

### DataSets

In Spark, datasets are an extension of dataframes. Basically, it earns two different APIs characteristics, such as strongly typed and untyped. Datasets are by default a collection of strongly typed JVM objects

**data frames** allow to deal with data in structured way. We will see how the data frame abstraction, very popular in other data analytics ecosystems (e.g. R and Python/Pandas), it is very powerful when performing exploratory data analysis. In fact, it is very easy to express data queries when used together with the SQL language

#### Creating a RDD from a file
The most common way of creating an RDD is to load it from a file. Notice that Spark's textFile can handle compressed files directly.

In [13]:
raw_data = sc.textFile('otomoto.csv').cache()
csv_data = raw_data.map(lambda l: l.split(";"))
header = csv_data.first() #extract header
data = csv_data.filter(lambda r: r != header)

In [16]:
list(enumerate(header))

[(0, '"id"'),
 (1, '"url"'),
 (2, '"Faktura VAT"'),
 (3, '"Marka pojazdu"'),
 (4, '"Model pojazdu"'),
 (5, '"Kategoria"'),
 (6, '"Pojemność skokowa"'),
 (7, '"Liczba miejsc"'),
 (8, '"Zarejestrowany w Polsce"'),
 (9, '"Wersja"'),
 (10, '"Typ"'),
 (11, '"Rok produkcji"'),
 (12, '"Serwisowany w ASO"'),
 (13, '"Leasing"'),
 (14, '"VIN"'),
 (15, '"Homologacja ciężarowa"'),
 (16, '"VAT marża"'),
 (17, '"Pierwszy właściciel"'),
 (18, '"Filtr cząstek stałych"'),
 (19, '"Rodzaj paliwa"'),
 (20, '"Emisja CO2"'),
 (21, '"Kraj pochodzenia"'),
 (22, '"Numer rejestracyjny pojazdu"'),
 (23, '"Bezwypadkowy"'),
 (24, '"Stan"'),
 (25, '"Pierwsza rejestracja"'),
 (26, '"title"'),
 (27, '"price"'),
 (28, '"Skrzynia biegów"'),
 (29, '"Perłowy"'),
 (30, '"Kierownica po prawej (Anglik)"'),
 (31, '"Liczba drzwi"'),
 (32, '"Metalik"'),
 (33, '"Przebieg"'),
 (34, '"Akryl (niemetalizowany)"'),
 (35, '"Możliwość finansowania"'),
 (36, '"Uszkodzony"'),
 (37, '"Oferta od"'),
 (38, '"Kod Silnika"'),
 (39, '"Kolor"'

In [21]:
from pyspark.sql import Row
row_data = data.map(lambda p: Row(
    _id=(p[0]),
    marka=(p[3]),
    model=(p[4]),
    price=(p[27]),
    przebieg=(p[33])
    )
)

Once we have our RDD of Row we can infer and register the schema.

In [28]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [29]:
DF = sqlContext.createDataFrame(row_data)

In [30]:
DF.take(1)

[Row(_id='"6039269187"', marka='"Renault"', model='"Fluence"', price='"25900.0"', przebieg='"172 000 km"')]

https://spark.apache.org/docs/latest/sql-data-sources-load-save-functions.html

In [32]:
DF = spark.read.load("otomoto.csv", format="csv", sep=";", inferSchema="true", header="true")

In [34]:
DF.take(1)

[Row(id=6039269187, url='https://www.otomoto.pl/oferta/renault-fluence-bezwypadkowy-instalacja-gazowa-ID6AI9Nx.html', Faktura VAT=None, Marka pojazdu='Renault', Model pojazdu='Fluence', Kategoria='Osobowe', Pojemność skokowa='1 600 cm3', Liczba miejsc=5, Zarejestrowany w Polsce=None, Wersja=None, Typ='Sedan', Rok produkcji=2013, Serwisowany w ASO=None, Leasing=None, VIN=None, Homologacja ciężarowa=None, VAT marża=None, Pierwszy właściciel='Tak', Filtr cząstek stałych=None, Rodzaj paliwa='Benzyna+LPG', Emisja CO2=None, Kraj pochodzenia='Czechy', Numer rejestracyjny pojazdu=None, Bezwypadkowy='Tak', Stan='Używane', Pierwsza rejestracja='8 listopad 2013', title='Renault Fluence', price=25900.0, Skrzynia biegów='Manualna', Perłowy=None, Kierownica po prawej (Anglik)=None, Liczba drzwi=4, Metalik=None, Przebieg='172 000 km', Akryl (niemetalizowany)='Tak', Możliwość finansowania=None, Uszkodzony=None, Oferta od='Osoby prywatnej', Kod Silnika=None, Kolor='Biały', Napęd='Na przednie koła', Moc

In [35]:
type(DF)

pyspark.sql.dataframe.DataFrame

In [36]:
DF.registerTempTable("cars")

In [72]:
volvos = sqlContext.sql("""
    Select `Marka pojazdu` as marka, `Model pojazdu` as model, `price`, `przebieg`, `Rok produkcji` as rok  from cars 
    WHERE `Marka pojazdu` = "Volvo"
    AND `Model pojazdu` = "XC 60"
""")

In [73]:
type(volvos)

pyspark.sql.dataframe.DataFrame

The results of SQL queries are RDDs and support all the normal RDD operations.

In [74]:
volvos.take(1)

[Row(marka='Volvo', model='XC 60', price=89900.0, przebieg='132 789 km', rok=2016)]

In [75]:
volvos.show()

+-----+-----+--------+----------+----+
|marka|model|   price|  przebieg| rok|
+-----+-----+--------+----------+----+
|Volvo|XC 60| 89900.0|132 789 km|2016|
|Volvo|XC 60|121951.0| 25 236 km|2016|
|Volvo|XC 60| 54900.0|221 000 km|2012|
|Volvo|XC 60| 79999.0|140 515 km|2012|
|Volvo|XC 60| 89900.0|159 900 km|2012|
|Volvo|XC 60| 98900.0| 22 739 km|2016|
|Volvo|XC 60| 79555.0| 62 321 km|2012|
|Volvo|XC 60| 52900.0| 60 000 km|2014|
|Volvo|XC 60| 66900.0| 15 000 km|2015|
|Volvo|XC 60| 59900.0|183 000 km|2010|
|Volvo|XC 60|112800.0| 32 000 km|2016|
|Volvo|XC 60|123000.0| 21 000 km|2016|
|Volvo|XC 60| 87000.0|144 000 km|2015|
|Volvo|XC 60| 88900.0| 66 200 km|2014|
|Volvo|XC 60|185900.0|  3 000 km|2017|
|Volvo|XC 60| 75900.0|132 000 km|2014|
|Volvo|XC 60| 43500.0|205 500 km|2010|
|Volvo|XC 60| 68000.0|110 094 km|2014|
|Volvo|XC 60|122900.0| 13 000 km|2017|
|Volvo|XC 60| 79900.0|145 000 km|2016|
+-----+-----+--------+----------+----+
only showing top 20 rows



In [76]:
for volvo in volvos.collect():
    print("{} for {} with millage {}".format(volvo.model, volvo.price, volvo.przebieg))

XC 60 for 89900.0 with millage 132 789 km
XC 60 for 121951.0 with millage 25 236 km
XC 60 for 54900.0 with millage 221 000 km
XC 60 for 79999.0 with millage 140 515 km
XC 60 for 89900.0 with millage 159 900 km
XC 60 for 98900.0 with millage 22 739 km
XC 60 for 79555.0 with millage 62 321 km
XC 60 for 52900.0 with millage 60 000 km
XC 60 for 66900.0 with millage 15 000 km
XC 60 for 59900.0 with millage 183 000 km
XC 60 for 112800.0 with millage 32 000 km
XC 60 for 123000.0 with millage 21 000 km
XC 60 for 87000.0 with millage 144 000 km
XC 60 for 88900.0 with millage 66 200 km
XC 60 for 185900.0 with millage 3 000 km
XC 60 for 75900.0 with millage 132 000 km
XC 60 for 43500.0 with millage 205 500 km
XC 60 for 68000.0 with millage 110 094 km
XC 60 for 122900.0 with millage 13 000 km
XC 60 for 79900.0 with millage 145 000 km
XC 60 for 94900.0 with millage 124 999 km
XC 60 for 54000.0 with millage 290 000 km
XC 60 for 71900.0 with millage 97 100 km
XC 60 for 87500.0 with millage 163 000 km

In [77]:
DF.printSchema()

root
 |-- id: long (nullable = true)
 |-- url: string (nullable = true)
 |-- Faktura VAT: string (nullable = true)
 |-- Marka pojazdu: string (nullable = true)
 |-- Model pojazdu: string (nullable = true)
 |-- Kategoria: string (nullable = true)
 |-- Pojemność skokowa: string (nullable = true)
 |-- Liczba miejsc: integer (nullable = true)
 |-- Zarejestrowany w Polsce: string (nullable = true)
 |-- Wersja: string (nullable = true)
 |-- Typ: string (nullable = true)
 |-- Rok produkcji: integer (nullable = true)
 |-- Serwisowany w ASO: string (nullable = true)
 |-- Leasing: string (nullable = true)
 |-- VIN: string (nullable = true)
 |-- Homologacja ciężarowa: string (nullable = true)
 |-- VAT marża: string (nullable = true)
 |-- Pierwszy właściciel: string (nullable = true)
 |-- Filtr cząstek stałych: string (nullable = true)
 |-- Rodzaj paliwa: string (nullable = true)
 |-- Emisja CO2: string (nullable = true)
 |-- Kraj pochodzenia: string (nullable = true)
 |-- Numer rejestracyjny poja

### Queries as DataFrame operations


In [78]:
DF.select("Marka pojazdu").groupBy("Marka pojazdu").count().show()

+-------------+-----+
|Marka pojazdu|count|
+-------------+-----+
|   Volkswagen| 5598|
|   Oldsmobile|    1|
|     Infiniti|   91|
|      Peugeot| 1906|
|          NSU|    1|
|        Lexus|  252|
|       Jaguar|  215|
|       Saturn|    1|
|          Żuk|    1|
|     Maserati|   43|
|        Rover|   32|
|        Aixam|   37|
|         Tata|    2|
|         Jeep|  426|
|       Lancia|   64|
|   Mitsubishi|  533|
|         Mini|  495|
|          Kia| 1061|
|         null|  470|
|    Chevrolet|  352|
+-------------+-----+
only showing top 20 rows



In [89]:
DF.select(DF["Model pojazdu"]) \
    .filter(DF["Marka pojazdu"]=="Volvo") \
    .groupBy("Model pojazdu") \
    .count().show()

+-------------+-----+
|Model pojazdu|count|
+-------------+-----+
|          S60|  130|
|          V90|   31|
|          850|    4|
|    Seria 400|    1|
|          V60|  112|
|          S80|   59|
|          745|    1|
|          S90|   48|
|          C30|   45|
|          V70|   74|
|        XC 40|   10|
|          V50|  147|
|          S40|   73|
|        XC 70|   37|
|    Seria 700|    1|
|        XC 60|  219|
|         Inny|    1|
|          C70|   10|
|    Seria 200|    1|
|          V40|  151|
+-------------+-----+
only showing top 20 rows



#### Creating and RDD using parallelize
Another way of creating an RDD is to parallelize an already existing list.

In [70]:
A = ((a, a*a) for a in range(1000))

In [71]:
data = sc.parallelize(A)

In [25]:
data.count()

1000

In [26]:
data.take(10)

[(0, 0),
 (1, 1),
 (2, 4),
 (3, 9),
 (4, 16),
 (5, 25),
 (6, 36),
 (7, 49),
 (8, 64),
 (9, 81)]